# Web scraping de literales con Python

In [17]:
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os

import time

import numpy as np
import pandas as pd

In [3]:
# Instantiate options
opts = Options()

# opts.add_argument(" — headless") # Uncomment if the headless version needed
opts.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set the location of the webdriver
chrome_driver = os.getcwd() + "\chromedriver.exe"



In [35]:
def getting_webs(sitemap):
    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    # Load the HTML page
    driver.maximize_window()
    driver.get(sitemap)
    #driver.implicitly_wait(220)
    #time.sleep(10)
    #javaScript = "window.scrollTo(0,document.body.scrollHeight);"
    #driver.execute_script(javaScript)
    #time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return list(map(lambda x: 'https://shop.mango.com' + x['href'], soup.find_all(lambda tag: tag.name == 'a' and 
                                   tag.get('class') == ['site-map-item'])))

web_list = getting_webs('https://shop.mango.com/es/sitemap')

In [72]:
re.findall(r'(\w+)/[\w-]+$',web_list[0])

['mujer']

In [87]:
def scrape_web(web):
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    driver.maximize_window()
    driver.get(web)
    driver.implicitly_wait(220)
    time.sleep(2)
    #Scroll infinito
    scroll_pause_time = 0.7 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 5

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 5
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break
    
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    sex = re.findall(r'(\w+)/[\w-]+$',web)[0]
    category = soup.find("h1", {"class":"sg-text-subtitle _1ZGtO"}).getText()
    names = soup.find_all("span", {"class":"product-name sg-text-body-small _2uGDp"})
    return (sex, category,list(map(lambda x: x.text, names)))

In [88]:
data = pd.DataFrame(columns=['Sex','Category','Literal'])
for web in web_list:
    sex, cat,items = scrape_web(web)
    web_data = pd.DataFrame({'Sex':sex, 'Category':cat,'Literal':items})
    data = data.append(web_data)

In [95]:
#pd.set_option('display.max_rows',None)
data.to_excel('literales_mango.xlsx')